<a href="https://colab.research.google.com/github/fayazmalik503/Recommendation_Algorithm_Based_on_Blending_Learning/blob/main/Recommendation_Algorithm_Based_on_Blending_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Recommendation Algorithm Based on Blending Learning